# Vanilla LSTM for Gene/No gene classification
The Milestone 1 corresponds to the classication task of, given a sequence, predict if it contains a gene, a partial sequence of a gene or just intergenic code.

In [1]:
!pip3 install pyfastx

In [0]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import pandas as pd

from tqdm import tqdm # progress bar
from preproc_pipeline import window_pipeline
from warnings import simplefilter

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

## 1. Dataset for training
The genome of E. coli will be used for this purpose.

In [0]:
simplefilter("ignore")
genome = "GCF_000008865.2_ASM886v2_genomic.fna"
feature_table = "GCA_000008865.2_ASM886v2_feature_table.tsv"
df = window_pipeline(genome, feature_table)

In [5]:
print(df.sequence.apply(lambda x: len(x)).max())
len(df)

50


36522

In [6]:
print(
    f"columns -> {list(df.columns)}\n"
    f"labels in dataframe -> {list(df.label.unique())}"
)

columns -> ['sequence', 'label']
labels in dataframe -> ['gene', 'intergenic', 'partial']


Let's get a one hot mapping for the labels.

In [7]:
labels = list(df.label.unique())
lab0 = np.zeros(len(labels))
lab2vec = {}
vec2lab = {}
for i, label in enumerate(list(df.label.unique())):
    labv = lab0.copy()
    labv[i] = 1
    lab2vec[label] = labv
    vec2lab[tuple(labv)] = label

print(f"lab2vec -> {lab2vec}\nvec2lab -> {vec2lab}")

lab2vec -> {'gene': array([1., 0., 0.]), 'intergenic': array([0., 1., 0.]), 'partial': array([0., 0., 1.])}
vec2lab -> {(1.0, 0.0, 0.0): 'gene', (0.0, 1.0, 0.0): 'intergenic', (0.0, 0.0, 1.0): 'partial'}


In [8]:
print(df[df.sequence.apply(lambda x: len(x)==0)].count())
df = df[~df.sequence.apply(lambda x: len(x)==0)]

sequence    0
label       0
dtype: int64


Need to check why it always generate a 0 length row. I think is the last one, but I am not sure.

In [9]:
df["label_onehot"] = df.label.apply(lambda x: lab2vec[x])
toy = pd.concat([df[df.label=="gene"].sample(n=1000),
                df[df.label=="intergenic"].sample(n=1000)])
                # df[df.label=="partial"].sample(n=100)]).reset_index(drop=True)
toy["label_onehot"] = toy.label_onehot.apply(lambda x: x[0:2])
df_train = toy.sample(frac=8/10) # shuffle
#df_test = df[~df.index.isin(df_train.index)].dropna().reset_index(drop=True)
df_test = toy[~toy.sequence.isin(df_train.sequence)].reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_train

,sequence,label,label_onehot
0,TAGTCAGAGTTTCTTGCTAACCAGCGACGCGCGCCAGATTCGGTTT...,gene,"[1.0, 0.0]"
1,TCATTGCCGTCACTGAAAGTGCTGCCGCCAGCAGCATTTTGCGCAT...,gene,"[1.0, 0.0]"
2,TGCGAATATTTGTCTACGGCAGTTTACGCCACAAACAAGGCAACAG...,gene,"[1.0, 0.0]"
3,AGACTATCTGAAAGTGTCCCATGTATGCCCCATAAGAAAGCAGGGT...,intergenic,"[0.0, 1.0]"
4,AGATGGTTGGATTATTGCTGTTAGTGAAGCTGACTTTATTATTCAT...,gene,"[1.0, 0.0]"
...,...,...,...
1595,CGCTAATCACTTCGCCAACGGCGATGCTGCTGGCATCTTTGCCTAA...,gene,"[1.0, 0.0]"
1596,TGTTATCCGCCAGTGTGAGCAAACCGCATTTTTCAGCAAAATATTC...,intergenic,"[0.0, 1.0]"
1597,TATTGAATGATGCCGCAGGCCATTCTCGCACCACCACCGCCCAGGG...,gene,"[1.0, 0.0]"
1598,GAATGGTTTGCAGCTCATTTACTGCTTCATCAACGAAAATTTTATC...,gene,"[1.0, 0.0]"


In [10]:
df_test = toy[~toy.sequence.isin(df_train.sequence)].reset_index(drop=True)
df_test

,sequence,label,label_onehot
0,CAGCGCACAACCAGCACGGAGCACTCTGCGTGACGCACTACAGCTG...,gene,"[1.0, 0.0]"
1,GGCGATCGCACCACGCCTTCTATCATTGCCTATACCCAGGATGGTG...,gene,"[1.0, 0.0]"
2,ACTGACGTTTTGGCGCGTACATGGGGTTGCACGGCTCTGTTCCATA...,gene,"[1.0, 0.0]"
3,TGCGTCGTTTATTGTCTGTCGCACCTGTCCTTCTCTGGCTGATTAC...,gene,"[1.0, 0.0]"
4,GCGGTGAAAGCGAACTCGGTTGTTTATTCAATTTTTTACTCTGTCC...,gene,"[1.0, 0.0]"
...,...,...,...
393,CGCTGTTTTTGTTTTCCTTTTGGATTAATTTCAGCGTATAATGCGC...,intergenic,"[0.0, 1.0]"
394,TCAGCATTAGCAAGCCCTGTTCCCACAAGAGACAGGGACAAGCACG...,intergenic,"[0.0, 1.0]"
395,GCTCATGGTATAATTTTTAAGACATGATCAAGAAGCACGTCTATTG...,intergenic,"[0.0, 1.0]"
396,TCTGGTACGCCTGGCAGATATTTTGCCTGCCGGGCTAACAGTGTGA...,intergenic,"[0.0, 1.0]"


## 2. Embeddings
The next step is to use the whole sequence to compute the embeddings. First, get a set of k-mers, that will be our words for this NLP problem.

In [0]:
def window(fseq, window_size, slide = 1):
    # create a window of size k
    N = len(fseq)
    for i in range(0, N - window_size + 1, slide):
      if i+window_size+slide < N:
        yield fseq[i:i+window_size]


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]

class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, padding_idx):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, 
                                       padding_idx=padding_idx) #used predefined nn.Embedding
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)
    
k = 3                     # so our words has length 3
SLIDE = 1                 # sampling slide
CONTEXT_SIZE = 2          # 2 words to the left, 2 to the right
EMDEDDING_DIM = 25        # embedding dimension
EPOCHS = 30               # number of epochs for training
model = None              # CBOW model

Finally, gather all the kmers and apply the CBOW algorithm.

In [0]:
# Since we have the embeddings stored, we are going to ignore the following next
# two cells and use this one
stored = False

if stored:
  with open("wti.p", "rb") as f:
    word_to_ix = pickle.load(f)
  ix_to_word = {v: k for k,v in word_to_ix.items()}
  model_save_name = 'ma_model.pt'
  path = model_save_name
  #path = F"/content/drive/My Drive/{model_save_name}" 
  model = CBOW(len(word_to_ix), EMDEDDING_DIM, padding_idx=word_to_ix["X"]).cpu()
  model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

In [13]:
# kmer in all of the 1/4 sequences of the dataset
kmers = [ kmer for kmer in window("".join([seq for seq in df_train.loc[:,"sequence"]]), k, SLIDE) ]  
vocab_size = len(kmers)

data = []
print("Filling context data...")
pbar = tqdm(total=vocab_size - 4-1) # just to output something in screen
for i in range(2, vocab_size - 2): # first word to have 2 words before is the "third" one (0,1,2)
    context = (kmers[i - 2], kmers[i - 1],
               kmers[i + 1], kmers[i + 2])
    target = kmers[i]
    data.append((context, target))
    pbar.update(1)

word_to_ix, ix_to_word = {},{}
for i, word in enumerate(set(kmers)):
    word_to_ix[word] = i
    ix_to_word[i] = word

ix = len(ix_to_word)
ix_to_word[ix] = "X"  # our padding character
word_to_ix["X"] = ix

with open("/content/drive/My Drive/wti.p", "wb") as f:
  pickle.dump(word_to_ix, f)

with open("/content/drive/My Drive/itw.p", "wb") as f:
  pickle.dump(ix_to_word, f)

  0%|          | 0/79991 [00:00<?, ?it/s]

Filling context data...


 27%|██▋       | 21343/79991 [00:00<00:00, 213423.59it/s]

In [14]:
import sys
from time import time 
model = CBOW(len(word_to_ix), EMDEDDING_DIM, padding_idx=word_to_ix["X"])

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
print("Training embeddings...")

model_save_name = 'ma_model.pt'
path = F"/content/drive/My Drive/{model_save_name}" 

start = time()
tot = len(data)
print()
for epoch in range(EPOCHS):
  start = time()
  total_loss = 0
  i = 1
  for context, target in data:
    if not i%100:
      sys.stdout.write(f"\r{i}/{tot} in {round(time()-start,2)}s        ")
      sys.stdout.flush()
    context_vector = make_context_vector(context, word_to_ix)  
    model.zero_grad()
    log_probs = model(context_vector)
    loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
    loss.backward()
    optimizer.step()
    total_loss += loss.data
    i+=1
  torch.save(model.state_dict(), path)
  print(f"\nEpoch {epoch+1}/{EPOCHS} in {round(time()-start,2)}\n\n")

Training embeddings...

27700/79992 in 19.74s        

79992it [00:20, 213423.59it/s]                           

79900/79992 in 56.32s        
Epoch 1/30 in 56.38836646080017


79900/79992 in 55.63s        
Epoch 2/30 in 55.700063705444336


79900/79992 in 58.94s        
Epoch 3/30 in 59.00810956954956


79900/79992 in 58.34s        
Epoch 4/30 in 58.411818742752075


79900/79992 in 57.09s        
Epoch 5/30 in 57.166804790496826


79900/79992 in 57.6s        
Epoch 6/30 in 57.665950298309326


79900/79992 in 56.33s        
Epoch 7/30 in 56.40046310424805


79900/79992 in 57.23s        
Epoch 8/30 in 57.31258416175842


79900/79992 in 56.69s        
Epoch 9/30 in 56.76210379600525


79900/79992 in 57.19s        
Epoch 10/30 in 57.25938630104065


79900/79992 in 57.71s        
Epoch 11/30 in 57.78800010681152


79900/79992 in 57.93s        
Epoch 12/30 in 58.00145125389099


79900/79992 in 56.46s        
Epoch 13/30 in 56.535449266433716


79900/79992 in 57.09s        
Epoch 14/30 in 57.15931820869446


79900/79992 in 56.69s        
Epoch 15/30 in 56.76209354400635


79900/79992 in 57.24s        


## 4. Vanilla RNN model

In [0]:
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, hidden_out, output_dim,
                 padding_idx, t):

        super().__init__()
        self.nb_tags = output_dim

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=padding_idx)
        
        self.rnn = nn.LSTM(embedding_dim, hidden_dim)
        
        self.lhid = nn.Linear(t*hidden_dim, hidden_out)
        # self.lhid = nn.Linear(hidden_dim*t, hidden_out)

        self.fc = nn.Linear(hidden_out, output_dim)


    def forward(self, text):

        #text = [sent len, batch size]
        # 1. embedding
        embedded = self.embedding(text)

        #embedded = [sent len, batch size, emb dim]
        output, (hidden, lt) = self.rnn(embedded)

        # 3. get that so it's correctly packed for the hidden layer
        output = output.contiguous()
        output = output.reshape(output.shape[0], -1)

        output = nn.functional.relu(self.lhid(output))
        #print('after linear layer')
        #print(output.size())

        # 4. classification
        output = self.fc(output)
        #print(output.size())
        output = nn.functional.log_softmax(output, dim=1)
        #print(output.size())
        # output = output.view(text.size()[0], -1)
        return output

## 4. Tweak the embeddings to accomodate varying sizes of the sequences
Once we have the model and the embeddings, we would need to tweak the embeddings so that they are adjusted for the padded sequences.

### 4.1. Add the padding char to the embeddings

Now, get sequences as indexes.

In [0]:
def to_ix(kmer, word_to_ix):
  if "X" in kmer:
    return word_to_ix["X"]
  elif kmer in word_to_ix:
    return word_to_ix[kmer]
  else:
    return len(word_to_ix)

def wind_idx(seq):
  return [to_ix(kmer, word_to_ix)  for kmer in window(seq, k, SLIDE)]

df_train["seq_idx"] = df_train["sequence"].apply(wind_idx)
df_test["seq_idx"] = df_test["sequence"].apply(wind_idx)

In [17]:
np.unique(df_train["seq_idx"].apply(lambda x: len(x)))

array([46])

Finally, instantiante the model and initialize the weigths of the embeddings.

In [18]:
print(word_to_ix["X"])
print(len(set(word_to_ix.keys()))-1)

64
64


In [0]:
t = len(df_train.iloc[0,0]) # Find length of seqs (fixed)

rnn = RNN(input_dim=len(word_to_ix), embedding_dim=EMDEDDING_DIM,
          hidden_dim = 34, hidden_out = 90, output_dim=2, 
          padding_idx = word_to_ix["X"], t = len(df_train.iloc[0,0]))

In [59]:
rnn.embedding.weight.data.copy_(model.embeddings.weight)

tensor([[-0.1137,  0.0999, -0.3476,  ...,  0.0821, -0.8537, -0.0803],
        [-0.8290,  0.5165, -0.1896,  ...,  1.7026,  2.1993,  0.1653],
        [ 0.1834,  0.8876, -0.3951,  ...,  1.8383, -1.1204, -0.7414],
        ...,
        [-0.5050, -0.7819, -0.1929,  ...,  0.8292, -0.1737,  0.9981],
        [-1.3405, -1.3242, -1.5161,  ..., -0.2734, -0.5583, -1.3304],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<CopyBackwards>)

## 5. Training loop


All that is left is to split our training and testing and train the model.

In [0]:
from torch.utils.data import DataLoader, Dataset

class oversampdata(Dataset):
  def __init__(self, data):
    # first column is list of index sentence
    self.data = torch.LongTensor(data.iloc[:,0])
    # second column is the label
    self.targets = torch.LongTensor(data.iloc[:,1])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    data_val = self.data[index]
    target = self.targets[index]
    return data_val, target

train_dataset = oversampdata(df_train.loc[:,["seq_idx", "label_onehot"]])
valid_dataset = oversampdata(df_test.loc[:,["seq_idx", "label_onehot"]])

In [0]:
BATCH_SIZE = 32

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, 
                                         shuffle=False)

In [0]:
def binary_accuracy(preds, y):
   """
   Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
   """
   rounded_preds = torch.zeros(preds.size())
   for i in range(len(preds)):
     idx_max = torch.where(preds[i].abs() == preds[i].abs().max())
     rounded_preds[i][idx_max] = 1
   rounded_preds = torch.FloatTensor(rounded_preds).int()
   correct = (rounded_preds == y).float() #convert into float for division
   acc = (correct.sum()/len(preds[0])) / len(correct)
   return acc

In [0]:
import sys

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    print("Training...")

    for i, batch in enumerate(iterator):
        if i%50:
          sys.stdout.write(f"\rIteration {i}        ")
          sys.stdout.flush()

        inputs, labels_onehot = batch

        optimizer.zero_grad()
                
        predictions = model(inputs)
        
        labels_idx = torch.LongTensor([np.where(label==1)[0][0] for label in labels_onehot])


        loss = criterion(predictions, labels_idx)
        
        loss.backward()
        
        optimizer.step()
        
        acc = binary_accuracy(predictions, labels_onehot)
        # print(loss.item())
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    print()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)



def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    print("Evaluating...")
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            inputs, labels_onehot = batch
            predictions = model(inputs)
            labels_idx = torch.LongTensor([np.where(label==1)[0][0] for label in labels_onehot])

            loss = criterion(predictions, labels_idx)
            acc = binary_accuracy(predictions, labels_onehot)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()

In [25]:
len(train_dataset)

1600

In [61]:
%%time
N_EPOCHS = 5
model_save_name="ma_rnn.pt"
path = F"/content/drive/My Drive/{model_save_name}"

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    print("Epoch: " + str(epoch))
    train_loss, train_acc = train(rnn, trainloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(rnn, testloader, criterion)

    #train_loss  = train(rnn, trainloader, optimizer, criterion)
    #valid_loss  = evaluate(rnn, testloader, criterion)    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(rnn.state_dict(), path)
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    
    #print(f'\tTrain Loss: {train_loss:.3f}')
    #print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 0
Training...
Iteration 49        
Evaluating...
	Train Loss: 0.693 | Train Acc: 49.88%
	 Val. Loss: 0.690 |  Val. Acc: 44.75%
Epoch: 1
Training...
Iteration 49        
Evaluating...
	Train Loss: 0.694 | Train Acc: 49.06%
	 Val. Loss: 0.690 |  Val. Acc: 44.75%
Epoch: 2
Training...
Iteration 49        
Evaluating...
	Train Loss: 0.694 | Train Acc: 50.69%
	 Val. Loss: 0.690 |  Val. Acc: 44.75%
Epoch: 3
Training...
Iteration 49        
Evaluating...
	Train Loss: 0.694 | Train Acc: 49.25%
	 Val. Loss: 0.690 |  Val. Acc: 44.75%
Epoch: 4
Training...
Iteration 49        
Evaluating...
	Train Loss: 0.694 | Train Acc: 50.00%
	 Val. Loss: 0.690 |  Val. Acc: 44.75%
CPU times: user 7min 11s, sys: 5.59 s, total: 7min 16s
Wall time: 16.5 s


In [62]:
# result = [(rnn(df_test.sequence.iloc[i,"sequence"]), df_test.label[i]) for i in range(20)]
# for r in result:
#   print(r)

result = [[rnn(torch.LongTensor(wind_idx(df_test.iloc[i,0])).unsqueeze(0)), df_test.label[i]] for i in range(20)]
for r in result:
  print(r)

[tensor([[-0.6874, -0.6989]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6958, -0.6905]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6810, -0.7054]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6628, -0.7244]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6640, -0.7232]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6595, -0.7279]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6762, -0.7104]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6958, -0.6905]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6710, -0.7158]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6848, -0.7016]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6818, -0.7046]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6588, -0.7288]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6623, -0.7250]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6740, -0.7127]], grad_fn=<LogSoftmaxBackward>), 'gene']
[tensor([[-0.6623, -0.7250]], grad